In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt

In [2]:
def open_file():
    with open('ddata_min.pickle', 'rb') as f:
        f_data = pickle.load(f)
    return f_data

In [3]:
def coin_select(name='BTC'):
    f_data = open_file()
    coin_name = name
    s_coin_data = f_data[f_data.permno == coin_name].copy()
    s_coin_data = s_coin_data.reset_index()
    
    return s_coin_data

In [4]:
def RSI(period=30, name='BTC'):
    s_coin_data = coin_select(name=name)
    period = period
    U = np.where(s_coin_data.prc.diff(1) > 0, s_coin_data.prc.diff(1), 0)
    D = np.where(s_coin_data.prc.diff(1) < 0, s_coin_data.prc.diff(1)*(-1), 0)
    AU = pd.DataFrame(U, index=s_coin_data.index).rolling(window=period).mean()
    AD = pd.DataFrame(D, index=s_coin_data.index).rolling(window=period).mean()
    RSI = AU / (AD+AU) * 100
    s_coin_data['RSI'] = RSI
    
    return s_coin_data['RSI']

In [42]:
def rsi_trading_test(initial_asset=100, sell_fee=0.0004, buy_fee=0.0004, name='BTC'):
    
    df_result = pd.DataFrame(columns=['Period', 'Final Asset', 'Winning', 'Losing'])
    s_coin_data = coin_select(name=name)
    
    for i in range(120, 201):
        period = i
        print('RSI period is {}'.format(period))
        
        s_coin_data['RSI'] = RSI(period=i, name=name)

        buy_price = 0
#         total_profit = 0
#         total_loss = 0
        win_count = 0
        lose_count = 0
        sell_signal = False
        asset = initial_asset
        sell_fee = sell_fee
        buy_fee = buy_fee
        position = False

        for i, row in s_coin_data.iterrows():
            date, price, rsi = row['date'], row['prc'], row['RSI']

            if sell_signal:
                # 수익률
                profit_pct = (price - buy_price) / buy_price - sell_fee
#                 s_coin_data.at[i, 'Profit'] = profit_pct * asset
                asset *= (1+profit_pct)

                #초기화
                buy_price = 0
                position = False
                sell_signal = False

                # 이익
                if profit_pct > 0:
                    win_count += 1
#                     total_profit += profit_pct * asset

                # 손실
                else:
                    lose_count += 1
#                     total_loss += profit_pct * asset


    #             print('SELL: {} | Sell price: {:7.2f} | Total asset: {:7.4f}'.format(date, price, asset), '\n')


            # Buy signal
            elif not(buy_price and position) and rsi <= 30:
                asset = asset * (1-buy_fee)
                buy_price = s_coin_data.prc[i]
                position = True

    #             print('BUY: {} | Buy price: {:7.2f} | Total asset: {:7.4f}'.format(date, buy_price, asset))

            # Sell signal
            elif (position and buy_price) and rsi >= 70:
                sell_signal = True

            elif asset <= 0:
                break

        df_result.loc[period] = [period, asset, win_count, lose_count]
        print('Final Asset: {:10.4f} | Winning Trade: {:7} | Losing Trade: {:7}'.
                    format(asset, win_count, lose_count))
    
    return df_result

In [7]:
df = open_file()

In [54]:
coin_list = df.permno.unique()
coin_list = np.delete(coin_list, 0)
coin_list

array(['ETH', 'BCH', 'XRP', 'EOS', 'LTC', 'TRX', 'ETC', 'LINK', 'XLM',
       'ADA', 'XMR', 'DASH', 'ZEC', 'XTZ', 'ATOM', 'BNB', 'ONT', 'IOTA',
       'BAT', 'VET', 'NEO', 'QTUM', 'IOST', 'THETA', 'ALGO', 'ZIL', 'KNC',
       'ZRX', 'COMP', 'OMG', 'DOGE', 'SXP', 'KAVA', 'BAND', 'RLC',
       'WAVES', 'MKR', 'SNX', 'DOT', 'DEFI', 'YFI', 'BAL', 'CRV', 'TRB',
       'YFII', 'SUSHI', 'RUNE', 'SRM', 'BZRX', 'EGLD', 'SOL', 'ICX',
       'STORJ', 'BLZ', 'UNI', 'AVAX', 'FTM', 'HNT', 'ENJ', 'FLM', 'TOMO',
       'REN', 'KSM', 'NEAR', 'FIL', 'AAVE', 'LRC', 'RSR', 'MATIC',
       'OCEAN', 'CVC', 'BEL', 'CTK', 'AXS', 'ALPHA', 'ZEN', 'SKL', 'GRT',
       '1INCH'], dtype=object)

In [55]:
result_df = pd.DataFrame(columns=['Period', 'Final Asset', 'Winning', 'Losing'])

for coin in coin_list:
    print('Coin: {}'.format(coin))
    rsi_coin = rsi_trading_test(name=coin)
    result_df = pd.concat([result_df, rsi_coin])

Coin: ETH
RSI period is 120
Final Asset:   150.5075 | Winning Trade:      13 | Losing Trade:       4
RSI period is 121
Final Asset:   150.4674 | Winning Trade:      12 | Losing Trade:       4
RSI period is 122
Final Asset:   144.3070 | Winning Trade:      13 | Losing Trade:       4
RSI period is 123
Final Asset:   295.9214 | Winning Trade:      12 | Losing Trade:       2
RSI period is 124
Final Asset:   192.3825 | Winning Trade:      11 | Losing Trade:       3
RSI period is 125
Final Asset:   179.6396 | Winning Trade:      10 | Losing Trade:       3
RSI period is 126
Final Asset:   313.2320 | Winning Trade:      10 | Losing Trade:       2
RSI period is 127
Final Asset:   252.2451 | Winning Trade:       8 | Losing Trade:       2
RSI period is 128
Final Asset:   315.2678 | Winning Trade:       9 | Losing Trade:       2
RSI period is 129
Final Asset:   273.7846 | Winning Trade:      10 | Losing Trade:       2
RSI period is 130
Final Asset:   200.7670 | Winning Trade:      10 | Losing Trad

KeyboardInterrupt: 